### Samples of fetching market data from EnergyDesk

This first section loads a token from a local .env file 

In [8]:
# Import relevant libraries
import sys,os
from os.path import join, dirname
from dotenv import load_dotenv

sys.path.insert(1, os.path.join(sys.path[0], '..'))  # Hack to include parent directory in search path

# Loading token, url etc from .env file in current directory
load_dotenv('.env')
token = os.environ.get("basic_auth_token")
server_url = os.environ.get("server_url")
print("Using server ", server_url)

Using server  https://dev.heco.energydesk.no


### Fetching Nasdaq OMX data

In [12]:
from marketdata.derivatives  import Derivatives
df_no1_epads=Derivatives.fetch_product_prices(server_url, token, "Nasdaq OMX", "Nordic Power", "NO3")
df_no1_epads

,ticker,instrument,delivery_from,delivery_until,timestamp,high,low,open,last,close,prev_close,trading_date,area
0,SYTRHAFUTBLMJUN-22,EPAD,2022-06-01,2022-07-01,2022-06-13,0.0,0,-75.37,-75.37,-75.37,-75.37,2022-06-13,NO3
1,SYTRHAFUTBLMJUL-22,EPAD,2022-07-01,2022-08-01,2022-06-13,0.0,0,-50.88,-50.88,-50.88,-50.88,2022-06-13,NO3
2,SYTRHAFUTBLMAUG-22,EPAD,2022-08-01,2022-09-01,2022-06-13,0.0,0,-60.88,-60.88,-60.88,-60.88,2022-06-13,NO3
3,SYTRHFUTBLQ3-22,EPAD,2022-07-01,2022-10-01,2022-06-13,-64.5,-66,-66.00,-65.00,-65.00,-65.00,2022-06-13,NO3
4,SYTRHFUTBLQ4-22,EPAD,2022-10-01,2023-01-01,2022-06-13,0.0,0,-72.50,-72.50,-72.50,-72.50,2022-06-13,NO3
5,SYTRHFUTBLQ1-23,EPAD,2023-01-01,2023-04-01,2022-06-13,0.0,0,-79.25,-79.25,-79.25,-79.25,2022-06-13,NO3
6,SYTRHFUTBLYR-23,EPAD,2023-01-01,2024-01-01,2022-06-13,0.0,0,-46.00,-46.00,-46.00,-46.00,2022-06-13,NO3
7,SYTRHFUTBLYR-24,EPAD,2024-01-01,2025-01-01,2022-06-13,0.0,0,-25.40,-25.40,-25.40,-25.40,2022-06-13,NO3
8,SYTRHFUTBLYR-25,EPAD,2025-01-01,2026-01-01,2022-06-13,0.0,0,-20.25,-20.25,-20.25,-20.25,2022-06-13,NO3


### Fetching daily prices (generated from products)

In [19]:
current_area="NO1"
df_daily_prices=Derivatives.fetch_daily_prices(server_url, token, "Nasdaq OMX", "Nordic Power", current_area)
df_daily_prices.rename(columns = {'price': current_area + '(EUR)'}, inplace = True)
df_daily_prices.index=df_daily_prices['date']
df_daily_prices.drop('date', axis=1, inplace=True)  # No need for this extra column

# Create NOK prices (for simplification use spot crossrates)
from marketdata.moneymarkets import MoneyMarkets
df_forex=MoneyMarkets.fetch_forex_spot_rates(server_url, token)
eurnok=10 if df_forex is None or len(df_forex.index)==0 else df_forex["EURNOK"].values[-1]  #Pick latest EURNOK
df_daily_prices[ current_area + '(NOK)']=df_daily_prices[ current_area + '(EUR)'] * eurnok
df_daily_prices

,EUR,NOK
date,,
2022-06-01,135.68,1409.281024
2022-06-02,135.68,1409.281024
2022-06-03,135.68,1409.281024
2022-06-04,135.68,1409.281024
2022-06-05,135.68,1409.281024
...,...,...
2032-12-28,63.15,655.926420
2032-12-29,63.15,655.926420
2032-12-30,63.15,655.926420
